In [ ]:
BRANCH='prompt-learning-documentation'

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
#!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

# Introduction

In this notebook we demonstrate how to use p-tunining and prompt tuning within NeMo-Megatron. Both methods are parameter efficient alternatives to fine-tuning pretrained language models. Our NeMo implementation makes it possible to use one pretrained GPT model on many downstream tasks without needing to tune the model's full set of parameters. It also allows for adding new tasks to your model without overwriting or disrupting previous tasks on which the model has already been p-tuned/prompt-tuned. Because the original model parameters are frozen and never altered by either method, p-tuning/prompt-tuning also avoid cartographic forgetting issues often encountered when fine-tuning models. 

- Our prompt tuning implementation is based off Lester et. al’s EMNLP 2021 paper [The Power of Scale for Parameter-Efficient Prompt Tuning](https://arxiv.org/abs/2104.08691)

- Our p-tuning implementation is based off Liu et al's paper [GPT Understands, Too](https://arxiv.org/abs/2103.10385).

- Usage examples and API documentation can be found in [our user docs](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/megatron.html#prompt-tuning). 

<img src="images/prompt_learning_forward_pass.png" alt="Prompt Learning Forward Pass"/>


# Techincal Overview
Instead of selecting discrete text prompts in a manual or automated fashion, prompt-tuning and p-tuning utilize virtual prompt tokens that can be optimized via gradient decent. The only difference between prompt-tuning and p-tuning within NeMo-Megatron is the architecture used to tune the soft prompt tokens during training. 

### Terminology
We will be using the terms `continuous`, `soft`, and `virtual` interchangeably to refer to tokens inserted into the model prompt that have *no* concrete mapping to strings or characters within the model’s vocabulary. These virtual tokens exist in contrast to `discrete`, `hard`, or `real` tokens that *do* make up the model’s vocabulary.  Virtual tokens are purely 1D vectors with dimensionality equal to that of each real token embedding matching the `hidden_size` hyperparameter. In training and inference, continuous token embeddings are inserted amoung discrete token embeddings according to a template you provide in the model's config. We will show how to do this in the code below. 

When referring to p-tuning and prompt tuning together, we will be using the phrase `prompt learning` for simplicity.


### Prompt-Tuning
When prompt-tuning a pretrained GPT model, soft prompt embeddings are initialized as a 2D matrix of size `total_virtual_tokens X hidden_size`. Each task the model is prompt-tuned to perform has its own 2D embedding matrix associated with it. Tasks do not share any parameters during traning or inference. All GPT model parameters are frozen and only the embedding parameters for each task are updated during training. 

In prompt tuning you can specify how the embeddings are initialized for each task. You can either 

1. Initialize embedding parameters according to some random distribution
2. Initialize embedding parameters from existing vocabulary embeddings (recommended)

If you choose to initialize virtual token embeddings from existing embedding weights, you can provide the string of words you want to use for initialization in the model's config. This string will be tokenized and tiled or truncated to match the specified number of virtual tokens you would like to use (`total_virtual_tokens`). Vocab embeddings are copied and used to initialize the soft prompt embedding matrix for each task. The vocab embeddings themselves are not updated or changed during prompt tuning. 


### P-Tuning
When p-tuning, an LSTM model is used to predict virtual token embeddings. LSTM parameters are randomly initialized at the start of p-tuning. All GPT model parameters are frozen, and only the LSTM weights are updated at each training step. LSTM parameters are shared between all tasks that are p-tuned at the same time, but the LSTM model outputs unique virtual token embeddings for each task. The virtual tokens predicted by the LSTM are inserted among the discrete token input in the exact same manner as with prompt-tuning. You still specify the number of virtual tokens you want to use by setting `total_virtual_tokens` and each virtual token embedding is still a 1D vector of size `hidden_size`.


# The Best of Both
A single pretrained GPT model can use both p-tuning and prompt-tuning. While you must decide to use either p-tuning or prompt-tuning for each task you want your model to perform, you can p-tune your model on a set of tasks *A*, then prompt tune your same model on a different set of tasks *B* and run inference on tasks from both *A* and *B* at the same time. During prompt-tuning or p-tuning, tasks tuned at the same time must use the same number of virtual tokens. During inference, tasks using differing amounts of virtual tokens can be run at the same time. 

P-tuning usually requires fewer virtual tokens per task to achieve good results but uses a higher number of parameters (~ 6% original model params) compared to prompt-tuning (~ .06% of original model params). Because p-tuning shares parameters between tasks during training, p-tuning on many very different tasks at once might perform worse than prompt tuning, which tunes a distinct set of parameters for each task. For the same reason, p-tuning your model on multiple tasks that are similar might allow your model to share insight between tasks.


With all that covered, let's get started!


In [ ]:
import os
import wget 

# Tasks and Datasets
We will be using p-tuning and prompt tuning to teach our GPT model to do 3 tasks: **Sentiment Analysis**, **Question Answering** and **Intent and Slot Classification**.

We will use [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) for our sentiment analysis task, [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for question answering task, and the [Assistent Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) for intent and slot classification. 

- The [Financial PhraseBank dataset](https://huggingface.co/datasets/financial_phrasebank) contains the sentiments for financial news headlines from the perspective of a retail investor. Further details about the dataset can be found in Malo et. al's "[Good Debt or Bad Debt: Detecting Semantic Orientations in Economic Texts](https://arxiv.org/abs/1307.5336)"


- [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) is a reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".


- The [Assistent Benchmarking Dataset](https://github.com/xliuhw/NLU-Evaluation-Data) is a natural language dataset for in home human-robot interaction. Details on the dataset can be found in Liu et. al's "[Benchmarking Natural Language Understanding Services for building Conversational Agents](https://arxiv.org/abs/1903.05566)"

Each of these tasks require different types of natural language understanding and lie in different domains. We will demonstrate how to use one model to perform all of them.

# Data Preparation

The prompt learning dataset loader accepts a list of json/dictionary objects or a list of json file names where each json file contains a collection of json objects. Each json object must include the field `taskname` which is a string identifier for the task the data example corresponds to. They should also include one or more fields corresponding to different sections of the discrete text prompt. The input data might look like:

```
[
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT1], "question": [QUESTION_TEXT1], "answer": [ANSWER_TEXT1]},
    {"taskname": "squad", "context": [CONTEXT_PARAGRAPH_TEXT2], "question": [QUESTION_TEXT2], "answer": [ANSWER_TEXT2]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT1], "label": [INTENT_TEXT1][SLOT_TEXT1]},
    {"taskname": "intent_and_slot", "utterance": [UTTERANCE_TEXT2], "label": [INTENT_TEXT2][SLOT_TEXT2]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT1], "label": [SENTIMENT_LABEL1]},
    {"taskname": "sentiment", "sentence": [SENTENCE_TEXT2], "label": [SENTIMENT_LABEL2]},
]
```

These additional fields can be unlimited in number and will be used to help map different parts of the discrete text input to a prompt template that you define. We will show how this mapping works and how to construct your prompt template in the `Prompt Formatting` section. 

In [ ]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = '.'

os.makedirs(DATA_DIR, exist_ok=True)


For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now. 

In [ ]:
# download the preprocessing scripts from github for the purpose of this tutorial
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/text_classification/data/prompt_learning_financial_phrase_bank_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/data/prompt_learning_squad_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/prompt_learning_assistant_preprocessing.py', NEMO_DIR)
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/data/assistant_utils.py', NEMO_DIR)

Now let's down load and process each dataset.

###  Financial PhraseBank Dataset

In [ ]:
# Download the financial phrase bank dataset
!wget https://www.researchgate.net/profile/Pekka_Malo/publication/251231364_FinancialPhraseBank-v10/data/0c96051eee4fb1d56e000000/FinancialPhraseBank-v10.zip
!unzip FinancialPhraseBank-v10.zip -d {DATA_DIR}

In [ ]:
# What the financial phrase bank dataset looks like before processing
SENTIMENT_DIR = os.path.join(DATA_DIR, "FinancialPhraseBank-v1.0")
!head -4 $SENTIMENT_DIR/Sentences_AllAgree.txt

In [ ]:
# Preprocess financial phrase bank dataset
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py

In [ ]:
# What the financial phrase bank dataset looks like after processing
!head -4 $SENTIMENT_DIR/financial_phrase_bank_train.jsonl

Our financial phrase bank preprocessing script converted the raw text file of sentences and labels into three `.jsonl` files for training, validation, and testing. Each line in the files contains a json object with the fields `taskname`, `sentiment`,`sentence`, and `label`. You can inspect the preprocessing script and play with different arguments for the script by looking at and running `prompt_learning_financial_phrase_bank_preprocessing.py` which should currently be downloaded in `NEMO_DIR`. It is also located at `examples/nlp/text_classification/data/prompt_learning_financial_phrase_bank_preprocessing.py` in the NeMo repo.

By default 80% of the data was randomly selected for the training set, 10% for the validation set, and 10% for the test set. We only used training examples with 100% agreement from labelers on the correct sentiment label. This data is from `Sentences_AllAgree.txt`. This should result in `1811` training examples, `226` validation examples, and `227` examples for testing. The `label` field was removed from test examples. 

If you want to try using more data, you can combine the `Sentences_AllAgree.txt` with any of the `Sentences_75Agree.txt`, `Sentences_66Agree.txt` and/or `Sentences_50Agree.txt` by creating a new cell below and running:

```
!cat {SENTIMENT_DIR}/Sentences_AllAgree.txt {SENTIMENT_DIR}/Sentences_75Agree.txt >> {SENTIMENT_DIR}/combined_data.txt
!python $NEMO_DIR/prompt_learning_financial_phrase_bank_preprocessing.py --file-name combined_data.txt
```


### SQuAD Dataset

In [ ]:
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

# Download the SQuAD dataset
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!mv train-v2.0.json {SQUAD_DIR}

In [ ]:
# Preprocess squad data
!python $NEMO_DIR/prompt_learning_squad_preprocessing.pyy --data-dir {SQUAD_DIR}

In [ ]:
# What the squad dataset looks like after processing
!head -4 $SQUAD_DIR/squad_train.jsonl

We made a `.jsonl` file for each of the train, validation, and testing splits of the squad data. Every `.jsonl` file contains json objects with the fields `taskname`, `context`, `question`, and `answer`. The preprocessing script is called `prompt_learning_squad_preprocessing.py`. It should be in your `NEMO_DIR` and at `examples/nlp/question_answering/data/prompt_learning_squad_preprocessing.py` in the NeMo repo. 

The SQuAD dataset consists of various topics like `Beyoncé`, `IPod`, and `Symbiosis`. Each topic has several paragraphs associated with it, and each paragraph has several questions and answers related to it. When we separated the train/validation/test splits, we separated them on the topic level. For example, if the training set contains paragraphs and questions about the topic `Beyoncé`, neither the validation nor test sets will contain any questions on this topic. All questions about a certain topic are isolated to one split of the data. 

Like the Financial PhraseBank Dataset, we randomly selected 80% of the questions for training, 10% for validation, and 10% for test. This resulted in `69125` test examples, `8952` validation examples, and `8744` testing examples. The `answer` field was removed from test examples.

### Assistent Dataset

In [ ]:
ASSISTENT_DIR = os.path.join(DATA_DIR, "assistent")
os.makedirs(ASSISTENT_DIR, exist_ok=True)

# Download the assisent dataset
!wget https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip
!unzip master.zip -d {ASSISTENT_DIR}

In [ ]:
# Process virtual assistent intent and slot classification data
!python $NEMO_DIR/prompt_learning_assistant_preprocessing.py

In [ ]:
!head -5 $ASSISTENT_DIR/assistent_train.jsonl

For the virtual assistent dataset, there are a set of 64 possible intents:

In [ ]:
!echo 'Intents: ' $(wc -l < {ASSISTENT_DIR}/nemo-format/dict.intents.csv)
!cat {ASSISTENT_DIR}/nemo-format/dict.intents.csv

and 55 types of slots:

In [ ]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {ASSISTENT_DIR}/nemo-format/dict.slots.csv)
!cat {ASSISTENT_DIR}/nemo-format/dict.slots.csv

each slot label consists of the slot type followed by specific text from the utterance that corresponds to that slot type. For example, the utterance `"play real racing three game"` has the intent label `play_game` and the slot label `game_name(real racing three)`. Utterances each have one intent label and zero or more slot labels. In cases where there is no slot label, our GPT model should predict the word `None`. 

Json objects for each training example contain three fields: `taskname`, `utterance`, and `label`. For this dataset, our preprocessing scipt formatted our intent and slot labels to look like `"\nIntent: transport_taxi\nSlots: transport_agency(golden taxi), time(seven pm), date(today)"`. With newline characters (\n) separating intent and slot labels. Our train jsonl file has `9960` training examples. Our validation and test jsonl files each have `538` training examples. Test examples do not have the `label` field. 

# Model Config Setup

Now we will begin setting up the conifg file used for prompt/p-tuning our GPT models! GPT Prompt learning within NeMo uses a class called `MegatronGPTPromptLearningModel` which has its own config file. We will start by loading an example prompt learning config file, then make changes to it to fit our tasks and training plans. 

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/conf/megatron_gpt_prompt_learning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_prompt_learning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

First let's set the datasets we've created in the config. We are going to start by p-tuning a GPT model on the **Sentiment Analysis** and **Intent and Slot Classification** tasks. We will set only those two datasets in the config file right now. 

In [ ]:
config.model.data.train_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_train.jsonl", f"{ASSISTENT_DIR}/assistent_train.jsonl"]
config.model.data.validation_ds = [f"{SENTIMENT_DIR}/financial_phrase_bank_val.jsonl", f"{ASSISTENT_DIR}/assistent_val.jsonl"]

The `MegatronGPTPromptLearningModel` class expects datasets to be a list of `.json` or `.jsonl` file paths. You can give it multiple datasets at once like we did above.  

### Prompt Formatting
Now that we have our datasets set, lets define what we want the prompt for each task to look like. 

To customize different prompts for different tasks, we simply need to specify the prompt task template in the config file. The virtual token markers `<|VIRTUAL_PROMPT_#|>` signify where you want virtual tokens to be placed in the template string. `<|VIRTUAL_PROMPT_0|>`, `<|VIRTUAL_PROMPT_1|>`, and `<|VIRTUAL_PROMPT_2|>` indicate where a number of virtual tokens matching the values given at `virtual_token_splits[0]`, `virtual_token_splits[1]` and `virtual_token_splits[2]` will be placed. The other variable fields `{var}` refer to the fields in the data json. 

For example, given: 

1. the data json **{"sentence1": "And he said, Mama, I'm home.", "sentence2": "He didn't say a word."}**


2. virtual token splits set to `virtual_token_splits = [3, 3, 3]` 


3. a prompt template set to `prompt_template = "<|VIRTUAL_PROMPT_0|> Hypothesis: [sentence1], <|VIRTUAL_PROMPT_1|> Premise: [sentence2] <|VIRTUAL_PROMPT_2|> Answer:"`

the input will be translated into **<span style="color:red">VVV</span> Hypothesis: And he said, Mama, I'm home.<span style="color:red">VVV</span> Premise: He didn't say a word.<span style="color:red">VVV</span> Answer:**, where <span style="color:red">VVV</span> are three virtual tokens.

Let's configure a template for our **Sentiment Analysis** and **Intent and Slot Classification** tasks:


In [ ]:
config.model.task_templates = [
    {
        "taskname": "sentiment",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> {sentence} sentiment: {label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [10],
        "truncate_field": None
    },
    {
        "taskname": "intent_and_slot",
        "prompt_template": "<|VIRTUAL_PROMPT_0|> Predict intent and slot <|VIRTUAL_PROMPT_1|> {utterance}{label}",
        "total_virtual_tokens": 10,
        "virtual_token_splits": [5, 5],
        "truncate_field": None
    }
]

Note each `task_template` item has 5 fields. 

- **`prompt_template`** is a string showing the model where to place virtual tokens and how to map dataset json fields to where they belong in the model prompt. 


- **`taskname`** refers to the same `taskname` in the dataset json objects. 


- **`truncate_field`** specifies which field in the data json to truncate if the length of the input exceeds the maximum sequence length of the model. If `truncate_field` is set to `None`, examples that are too long are simply dropped from the dataset.


- **`total_virtual_tokens`** specifies the total number of virtual tokens that will be inserted into the model prompt.


- **`virtual_token_splits`** specifies the number of virtual tokens that belong at each `<|VIRTUAL_PROMPT_#|>` marker. `virtual_token_splits` values should add up to `total_virtual_tokens`. The number of `virtual_token_splits` should match the number of `<|VIRTUAL_PROMPT_#|>` markers. 

### Setting New Tasks
After you p-tune your model this time, you can always go back and p-tune or prompt-tune your model on more tasks without over writting the virtual prompts who've trained this time. You can also use a different number of `total_virtual_tokens` between each training session as long as tasks ptuned or prompt tuned at the same time have the same number of `total_virtual_tokens`. For this reason, when you ptune on a new task, you need to tell your model which of your tasks are new and which ones already exist (and thus you don't want to tune them). 

You do this by setting the `new_tasks` and `existing_tasks` values in the config file. Because we are ptuning a model with no existing tasks, you should set `existing_tasks=[]` and `new_tasks=['qa-task', 'sentiment-task']` as follows:

In [ ]:
config.model.existing_tasks = []
config.model.new_tasks = ['qa-task', 'sentiment-task']

After ptuning and/or prompt tuning is complete, you can run inference on all tasks at the same time, regradless of their `total_virtual_tokens` value.

### Setting The Pre-Trained GPT Model
We still need to set which GPT model we want to p-tune/prompt tune. Prompt learning methods work best with large GPT language models (5B or above). For the purposes of this tutorial, we are going to download a 345M parameter GPT model from NVIDIA NGC and convert it to `.nemo` format. We will be using the download and conversion script available at `examples/nlp/language_model/download_and_convert_megatron_gpt.py` 

In [ ]:
# Download the scipt to our NEMO working dir
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/download_and_convert_megatron_gpt.py', NEMO_DIR)